In [3]:
import sys
sys.path.append('../lib/BioInfer_software_1.0.1/')
from BIParser import BIParser

In [4]:
parser = BIParser()

In [5]:
with open('../data/BioInfer_corpus_1.1.1.xml','r') as f:
    parser.parse(f)

Maybe have candidates as tuples of tokens and do log likelihood loss? How to deal with non-contiguous entities?
With log likelihood loss, if you propose a candidate and it's wrong, you lose points, if you know a candidate exists and you didn't propose it, you lose points. But you don't necessarily need to assign probability to all the candidates, since the candidate set may be infinite

propose all entities
propose relations between entities


Limitation of the INN paper is that I don't think it handles non-contiguous entities and entity gold labels are available

In [6]:
texts = []
labels = []

for s in parser.bioinfer.sentences.sentences:
    tokens = [t.getText() for t in s.tokens]
    NER_labels = ['O' for t in s.tokens]

    nested_entities = []
    for e in s.entities:
        for n in e.nestedEntities:
            nested_entities.append(n)

    top_level_entities = []
    for e in s.entities:
        if e not in nested_entities:
            top_level_entities.append(e)

    for e in top_level_entities:
        prefix = 'B-'
        entity_type = e.type.name
        if 'RELATIONSHIP' not in entity_type:
            for t in e.subTokens:
                NER_labels[t.token.sequence] = prefix + entity_type
                prefix = 'I-'
    texts.append(tokens)
    labels.append(NER_labels)

In [7]:
labels

[[u'B-Individual_protein',
  'O',
  u'B-Function_property',
  u'I-Function_property',
  'O',
  'O',
  'O',
  'O',
  'O',
  u'I-Protein_complex',
  u'I-Protein_complex',
  u'I-Protein_complex',
  'O'],
 ['O',
  'O',
  u'B-Protein_complex',
  u'I-Protein_complex',
  u'I-Protein_complex',
  u'I-Protein_complex',
  u'I-Protein_complex',
  u'I-Protein_complex',
  u'I-Protein_complex',
  u'I-Protein_complex',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  u'B-Gene/protein/RNA',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  u'B-Gene/protein/RNA',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  u'B-Gene/protein/RNA',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  u'B-Gene/protein/RNA',
  'O'],
 [u'B-Amount_property',
  u'I-Amount_property',
  u'I-Amount_property',
  'O',
  'O',
  'O',
  'O',
  'O',
  u'B-Cell_component',
  u'I-Cell_compone

In [66]:
tags = list(set([l for x in labels for l in x]))

In [67]:
tags

[u'I-REGULATION',
 u'I-DEPHOSPHORYLATION',
 u'I-CONTROL',
 u'B-DOWNREGULATION',
 u'I-POLYMERIZATION',
 u'I-Analog',
 u'I-DNA_family_or_group',
 u'B-Location_property',
 u'B-Protein_family_or_group',
 u'B-Amount_property',
 u'B-Cell_line',
 u'I-DISRUPTION',
 u'I-Other_organic_compounds',
 u'B-Inorganic',
 u'I-Gene',
 u'B-LOCALIZATION',
 u'B-INHIBITION',
 u'I-Amount_property',
 u'B-MEDIATION',
 u'I-Protein_family_or_group',
 u'B-Process_entity',
 u'B-Multi-cell_organism',
 u'I-Cell_line',
 u'I-MODIFICATION',
 u'B-DEPOLYMERIZATION',
 u'B-EFFECT',
 u'I-ADDITION',
 u'B-ACTIVATION',
 u'B-Protein_complex',
 u'I-Process_entity',
 u'B-INITIATION',
 u'B-Gene',
 u'B-INTERACTION',
 u'B-DNA_family_or_group',
 u'B-STIMULATION',
 u'I-Homolog',
 u'I-Artificial',
 u'I-Peptide',
 u'I-INHIBITION',
 u'I-MODULATION',
 u'I-SUPPRESSION',
 u'I-LOCALIZATION',
 u'I-UPREGULATION',
 u'B-INACTIVATION',
 u'B-Antibody',
 u'I-REMOVAL',
 u'I-Antibody',
 u'B-Fusion_protein',
 u'B-DISRUPTION',
 u'B-REGULATION',
 u'I-CLE

In [53]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Model, Sequential

In [54]:
model = Sequential([TextVectorization(max_tokens=4900,),
                    Embedding(input_dim=4900,output_dim=32),
                    Bidirectional(LSTM(units=64))])
model = TimeDistributed(Dense(n_tags, activation="softmax"))(model) 

NameError: name 'n_tags' is not defined

In [48]:
model

In [49]:
model.predict(texts[0])

array([[ 0.00171443, -0.00395286, -0.00121076, ..., -0.00611349,
        -0.00207645, -0.00400433],
       [ 0.00171443, -0.00395286, -0.00121076, ..., -0.00611349,
        -0.00207645, -0.00400433],
       [ 0.00171443, -0.00395286, -0.00121076, ..., -0.00611349,
        -0.00207645, -0.00400433],
       ...,
       [ 0.00171443, -0.00395286, -0.00121076, ..., -0.00611349,
        -0.00207645, -0.00400433],
       [ 0.00171443, -0.00395286, -0.00121076, ..., -0.00611349,
        -0.00207645, -0.00400433],
       [-0.00042423,  0.00289883, -0.00170858, ...,  0.0089041 ,
        -0.00726694,  0.00103697]], dtype=float32)

In [31]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [32]:
tokenizer.fit_on_texts(texts)

In [33]:
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
from ke

In [97]:
f = s.formulas[0]

In [98]:
f.rootNode.entity.getText()

u'inhibits'

In [99]:
f.rootNode.predicate.name

u'SUPPRESS'

In [100]:
a1 = f.rootNode.arguments[0]

In [103]:
a1.getText()

u'alpha-catenin'

In [104]:
a2 = f.rootNode.arguments[1]

In [105]:
a2.getText()

u'beta-catenin signaling'

In [111]:
for f in s.formulas:
    print f.rootNode.entity.getText()
    for a in f.rootNode.arguments:
        print '\t' + a.getText()
        for a in a.arguments:
            print '\t\t' + a.getText()
        
    print '\n\n'

inhibits
	alpha-catenin
	beta-catenin signaling



inhibits by
	preventing: PREVENT
		alpha-catenin
		formation of complex: CONTAIN
	beta-catenin signaling



inhibits by
	preventing: PREVENT
		alpha-catenin
		formation of complex: CONTAIN
	beta-catenin signaling



